# Ownership and Moves


## Ownership

Rust’s `Box` type serves as another example of ownership. A `Box<T>` is a pointer to a value of type T stored on the heap. Calling `Box::new(v)` allocates some heap space, moves the value `v` into it, and returns a Box pointing to the heap space. Since a `Box` owns the space it points to, when the Box is dropped, it frees the space too.

In [3]:
{
    let point = Box::new((0.625, 0.5)); // point allocated here 
    let label = format!("{:?}", point); // label allocated here 
    assert_eq!(label, "(0.625, 0.5)");
};
// both dropped here

Just as variables own their values, structs own their fields, and tuples, arrays, and vectors own their elements:

In [5]:
struct Person { name: String, birth: i32 }
let mut composers: Vec<Person> = Vec::new();
composers.push(Person { name: "Palestrina".to_string(),
                        birth: 1525 });
composers.push(Person { name: "Dowland".to_string(),
                        birth: 1563 });
composers.push(Person { name: "Lully".to_string(),
                        birth: 1632 }); 
for composer in &composers {
    println!("{}, born {}", composer.name, composer.birth);
};

Palestrina, born 1525
Dowland, born 1563
Lully, born 1632


![image.png](asset/ch4/1.png)

Rust programs don’t usually explicitly drop values at all, in the way C and C++ pro‐ grams would use free and delete. The way to drop a value in Rust is to remove it from the ownership tree somehow: by leaving the scope of a variable, or deleting an element from a vector, or something of that sort. At that point, Rust ensures the value is properly dropped, along with everything it owns.

In a certain sense, Rust is less powerful than other languages: every other practical programming language lets you build arbitrary graphs of objects that point to each other in whatever way you see fit. But it is exactly because Rust is less powerful that the analyses the language can carry out on your programs can be more powerful. Rust’s safety guarantees are possible exactly because the relationships it may encounter in your code are more tractable. This is part of Rust’s “radical wager” we mentioned earlier: in practice, Rust claims, there is usually more than enough flexi‐ bility in how one goes about solving a problem to ensure that at least a few perfectly fine solutions fall within the restrictions the language imposes.

That said, the concept of ownership as we’ve explained it so far is still much too rigid to be useful. Rust extends this simple idea in several ways:
* You can move values from one owner to another. This allows you to build, rearrange, and tear down the tree.
* Very simple types like integers, floating-point numbers, and characters are excused from the ownership rules. These are called Copy types.
* The standard library provides the reference-counted pointer types `Rc` and `Arc`, which allow values to have multiple owners, under some restrictions.
* You can “borrow a reference” to a value; references are non-owning pointers, with limited lifetimes.

Each of these strategies contributes flexibility to the ownership model, while still upholding Rust’s promises. 

## Moves
In Rust, for most types, operations like assigning a value to a variable, passing it to a function, or returning it from a function don’t copy the value: they move it. The source relinquishes ownership of the value to the destination and becomes uninitialized; the destination now controls the value’s lifetime. Rust programs build up and tear down complex structures one value at a time, one move at a time.

You may be surprised that Rust would change the meaning of such fundamental operations; surely assignment is something that should be pretty well nailed down at this point in history. However, if you look closely at how different languages have chosen to handle assignment, you’ll see that there’s actually significant variation from one school to another. The comparison also makes the meaning and consequences of Rust’s choice easier to see.

Consider the following Python code:

In [9]:
// s = ['udon', 'ramen', 'soba'] 
// t=s
// u=s

Each Python object carries a reference count, tracking the number of values that are currently referring to it. So after the assignment to `s`, the state of the program looks like (note that some fields are left out).

![image.png](asset/ch4/2.png)

Since only `s` is pointing to the list, the list’s reference count is 1; and since the list is the only object pointing to the strings, each of their reference counts is also 1.

What happens when the program executes the assignments to `t` and `u`? Python implements assignment simply by making the destination point to the same object as the source, and incrementing the object’s reference count. So the final state of the program is something like

![image.png](asset/ch4/3.png)

Python has copied the pointer from s into t and u and updated the list’s reference count to 3. Assignment in Python is cheap, but because it creates a new reference to the object, we must maintain reference counts to know when we can free the value.

Now consider the analogous C++ code:

In [11]:
// using namespace std;
// vector<string> s = { "udon", "ramen", "soba" }; 
// vector<string> t = s;
// vector<string> u = s;

![image.png](asset/ch4/4.png)

Depending on the values involved, assignment in C++ can consume unbounded amounts of memory and processor time. The advantage, however, is that it’s easy for the program to decide when to free all this memory: when the variables go out of scope, everything allocated here gets cleaned up automatically.

In a sense, C++ and Python have chosen opposite trade-offs: Python makes assign‐ ment cheap, at the expense of requiring reference counting (and in the general case, garbage collection). C++ keeps the ownership of all the memory clear, at the expense of making assignment carry out a deep copy of the object. C++ programmers are often less than enthusiastic about this choice: deep copies can be expensive, and there are usually more practical alternatives.

So what would the analogous program do in Rust? Here’s the code:

In [16]:
let s: Vec<String> = vec!["udon".to_string(), "ramen".to_string(), "soba".to_string()]; 
let t: Vec<String> = s;
// let u: Vec<String> = s;

![image.png](asset/ch4/5.png)

What has happened here? The initialization let t = s; moved the vector’s three header fields from s to t; now t owns the vector. The vector’s elements stayed just where they were, and nothing happened to the strings either. Every value still has a single owner, although one has changed hands. There were no reference counts to be adjusted. And the compiler now considers s uninitialized.

So what happens when we reach the initialization let u = s;? This would assign the uninitialized value s to u. Rust prudently prohibits using uninitialized values, so the compiler rejects this code with error:
```Rust
error[E0382]: use of moved value: `s`
let s = vec!["udon".to_string(), "ramen".to_string(), "soba".to_string()];
    - move occurs because `s` has type `Vec<String>`,
      which does not implement the `Copy` trait
let t = s;
        - value moved here
let u = s;
        ^ value used here after move
```

Consider the consequences of Rust’s use of a move here. Like Python, the assignment is cheap: the program simply moves the three-word header of the vector from one spot to another. But like C++, ownership is always clear: the program doesn’t need reference counting or garbage collection to know when to free the vector elements and string contents.

The price you pay is that you must explicitly ask for copies when you want them. If you want to end up in the same state as the C++ program, with each variable holding an independent copy of the structure, you must call the vector’s clone method, which performs a deep copy of the vector and its elements:

In [31]:
#![allow(unused)]
let s: Vec<String> = vec!["udon".to_string(), "ramen".to_string(), "soba".to_string()]; 
let t: Vec<String> = s.clone();
let u: Vec<String> = s.clone();

### More Operations That Move
Rust applies move semantics to almost any use of a value. Passing arguments to functions moves ownership to the function’s parameters; returning a value from a function moves ownership to the caller. Building a tuple moves the values into the tuple. And so on.

You may now have better insight into what’s really going on in the examples we offered in the previous section. For example, when we were constructing our vector of composers, we wrote:
```Rust
struct Person { name: String, birth: i32 }
let mut composers = Vec::new();
composers.push(Person { name: "Palestrina".to_string(),
                        birth: 1525 });
```
This code shows several places at which moves occur, beyond initialization and assignment:

**Returning values from a function:** 
The call Vec::new() constructs a new vector and returns, not a pointer to the vector, but the vector itself: its ownership moves from Vec::new to the variable composers. Similarly, the to_string call returns a fresh String instance.

**Constructing new values:** 
The name field of the new Person structure is initialized with the return value of to_string. The structure takes ownership of the string.

**Passing values to a function:** 
The entire Person structure, not a pointer to it, is passed to the vector’s push method, which moves it onto the end of the structure. The vector takes owner‐ ship of the Person and thus becomes the indirect owner of the name String as well.

Moving values around like this may sound inefficient, but there are two things to keep in mind. First, the moves always apply to the value proper, not the heap storage they own. For vectors and strings, the value proper is the three-word header alone; the potentially large element arrays and text buffers sit where they are in the heap. Sec‐ ond, the Rust compiler’s code generation is good at “seeing through” all these moves; in practice, the machine code often stores the value directly where it belongs.


### Moves and Indexed Content
We’ve mentioned that a move leaves its source uninitialized, as the destination takes ownership of the value. But not every kind of value owner is prepared to become uninitialized. For example, consider the following code:

In [21]:
// Build a vector of the strings "101", "102", ... "105"
let mut v: Vec<String> = Vec::new(); 
for i in 101..106{
    v.push(i.to_string());
}
// Pull out random elements from the vector.
let third: String = v[2]; // error: Cannot move out of index of Vec 
let fifth: String = v[4]; // here too

Error: cannot move out of index of `Vec<String>`

Error: cannot move out of index of `Vec<String>`

For this to work, Rust would somehow need to remember that the third and fifth elements of the vector have become uninitialized, and track that information until the vector is dropped. In the most general case, vectors would need to carry around extra information with them to indicate which elements are live and which have become uninitialized. That is clearly not the right behavior for a systems programming language; a vector should be nothing but a vector.

It also makes a similar complaint about the move to fifth. In the error message, Rust suggests using a reference, in case you want to access the element without moving it. This is often what you want. But what if you really do want to move an element out of a vector? You need to find a method that does so in a way that respects the limitations of the type. Here are three possibilities:

In [34]:
// Build a vector of the strings "101", "102", ... "105"
let mut v: Vec<String> = Vec::new(); 
for i in 101..=105{
    v.push(i.to_string());
}
// 1. Pop a value off the end of the vector:
let fifth: String = v.pop().expect("vector empty!"); 
assert_eq!(fifth, "105");

// 2. Move a value out of a given index in the vector, 
// and move the last element into its spot:
let second: String = v.swap_remove(1);
assert_eq!(second, "102");

// 3. Swap in another value for the one we're taking out:
let third: String = std::mem::replace(&mut v[2], "substitute".to_string()); 
assert_eq!(third, "103");
// Let's see what's left of our vector.
assert_eq!(v, vec!["101", "104", "substitute"]);

Each one of these methods moves an element out of the vector, but does so in a way
that leaves the vector in a state that is fully populated, if perhaps smaller.

Collection types like Vec also generally offer methods to consume all their elements in a loop:

In [39]:
let v: Vec<String> = vec!["liberté".to_string(), 
             "égalité".to_string(),
             "fraternité".to_string()];
for mut s in v { 
    s.push('!');
    println!("{}", s);
};
// try uncomment below
// for s in v { 
//     println!("{}", s);
// };

liberté!
égalité!
fraternité!


If you do find yourself needing to move a value out of an owner that the compiler can’t track, you might consider changing the owner’s type to something that can dynamically track whether it has a value or not. For example, here’s a variant on the earlier example:

In [43]:
struct Person { name: Option<String>, birth: i32 }
let mut composers: Vec<Person> = Vec::new();
composers.push(Person { name: Some("Palestrina".to_string()),
                            birth: 1525 });

// You can’t do this:
// let first_name = composers[0].name;

// this works:
let first_name: Option<String> = std::mem::replace(&mut composers[0].name, None); 
assert_eq!(first_name, Some("Palestrina".to_string())); 
assert_eq!(composers[0].name, None);

The `replace` call moves out the value of `composers[0].name`, leaving None in its place, and passes ownership of the original value to its caller. In fact, using Option this way is common enough that the type provides a take method for this very purpose. You could write the preceding manipulation more legibly as follows:

In [44]:
struct Person { name: Option<String>, birth: i32 }
let mut composers: Vec<Person> = Vec::new();
composers.push(Person { name: Some("Palestrina".to_string()),
                            birth: 1525 });
let first_name: Option<String> = composers[0].name.take();
assert_eq!(composers[0].name, None);

## Copy Types: The Exception to Moves
The examples we’ve shown so far of values being moved involve vectors, strings, and other types that could potentially use a lot of memory and be expensive to copy. Moves keep ownership of such types clear and assignment cheap. But for simpler types like integers or characters, this sort of careful handling really isn’t necessary.

Compare what happens in memory when we assign a String with what happens when we assign an i32 value:
```Rust
let string1 = "somnambulance".to_string(); 
let string2 = string1;
let num1: i32 = 36; 
let num2 = num1;
```

![image.png](asset/ch4/6.png)

The standard `Copy` types include all the machine integer and floating-point numeric types, the char and bool types, and a few others. A tuple or fixed-size array of Copy types is itself a `Copy` type.

Only types for which a simple bit-for-bit copy suffices can be Copy. As we’ve already explained, `String` is not a `Copy` type, because it owns a heap-allocated buffer. For similar reasons, `Box<T>` is not `Copy`; it owns its heap-allocated referent. The `File` type, representing an operating system file handle, is not `Copy`; duplicating such a value would entail asking the operating system for another file handle. Similarly, the `MutexGuard` type, representing a locked mutex, isn’t `Copy`: this type isn’t meaningful to copy at all, as only one thread may hold a mutex at a time.

As a rule of thumb, any type that needs to do something special when a value is dropped cannot be Copy: a Vec needs to free its elements, a File needs to close its file handle, a `MutexGuard` needs to unlock its mutex, and so on. Bit-for-bit duplication of such types would leave it unclear which value was now responsible for the original’s resources.

What about types you define yourself? By default, `struct` and `enum` types are not Copy:

In [46]:
struct Label { number: u32 }
fn print(l: Label) { println!("STAMP: {}", l.number); }
let l = Label {number: 3};
print(l);
println!("My label number is: {}", l.number);

Error: borrow of moved value: `l`

Since `Label` is not Copy, passing it to print moved ownership of the value to the print function, which then dropped it before returning. But this is silly; a Label is nothing but a `u32` with pretensions. There’s no reason passing `l` to print should move the value.

But user-defined types being non-Copy is only the default. If all the fields of your struct are themselves `Copy`, then you can make the type `Copy` as well by placing the attribute `#[derive(Copy, Clone)]` above the definition, like so:

In [47]:
#[derive(Copy, Clone)]
struct Label { number: u32 }
fn print(l: Label) { println!("STAMP: {}", l.number); }
let l = Label {number: 3};
print(l);
println!("My label number is: {}", l.number);

STAMP: 3
My label number is: 3


With this change, the preceding code compiles without complaint. However, if we try this on a type whose fields are not all `Copy`, it doesn’t work.

Why aren’t user-defined types automatically Copy, assuming they’re eligible? Whether a type is Copy or not has a big effect on how code is allowed to use it: Copy types are more flexible, since assignment and related operations don’t leave the original unini‐ tialized. But for a type’s implementer, the opposite is true: Copy types are very limited in which types they can contain, whereas non-Copy types can use heap allocation and own other sorts of resources. So making a type Copy represents a serious commitment on the part of the implementer: if it’s necessary to change it to non-Copy later, much of the code that uses it will probably need to be adapted.

While C++ lets you overload assignment operators and define specialized copy and move constructors, Rust doesn’t permit this sort of customization. In Rust, every move is a byte-for-byte, shallow copy that leaves the source uninitialized. Copies are the same, except that the source remains initialized. This does mean that C++ classes can provide convenient interfaces that Rust types cannot, where ordinary-looking code implicitly adjusts reference counts, puts off expensive copies for later, or uses other sophisticated implementation tricks.

But the effect of this flexibility on C++ as a language is to make basic operations like assignment, passing parameters, and returning values from functions less predictable. For example, earlier in this chapter we showed how assigning one variable to another in C++ can require arbitrary amounts of memory and processor time. One of Rust’s principles is that costs should be apparent to the programmer. Basic operations must remain simple. Potentially expensive operations should be explicit, like the calls to clone in the earlier example that make deep copies of vectors and the strings they contain.

## Rc and Arc: Shared Ownership
The `Rc` and `Arc` types are very similar; the only difference between them is that an `Arc` is safe to share between threads directly—the name `Arc` is short for atomic reference count—whereas a plain `Rc` uses faster non-thread-safe code to update its reference count. If you don’t need to share the pointers between threads, there’s no reason to pay the performance penalty of an `Arc`, so you should use `Rc`; Rust will prevent you from accidentally passing one across a thread boundary. The two types are otherwise equivalent, so for the rest of this section, we’ll only talk about Rc.

Earlier we showed how Python uses reference counts to manage its values’ lifetimes. You can use `Rc` to get a similar effect in Rust. Consider the following code:

In [48]:
use std::rc::Rc;
// Rust can infer all these types; written out for clarity
let s: Rc<String> = Rc::new("shirataki".to_string()); 
let t: Rc<String> = s.clone();
let u: Rc<String> = s.clone();

For any type `T`, an `Rc<T>` value is a pointer to a heap-allocated `T` that has had a reference count affixed to it. Cloning an `Rc<T>` value does not copy the `T`; instead, it simply creates another pointer to it and increments the reference count.

![image.png](asset/ch4/7.png)

Each of the three `Rc<String>` pointers is referring to the same block of memory, which holds a reference count and space for the `String`. The usual ownership rules apply to the Rc pointers themselves, and when the last extant Rc is dropped, Rust drops the `String` as well.
You can use any of `String`’s usual methods directly on an `Rc<String>`:

In [49]:
assert!(s.contains("shira"));
assert_eq!(t.find("taki"), Some(5));
println!("{} are quite chewy, almost bouncy, but lack flavor", u);

shirataki are quite chewy, almost bouncy, but lack flavor


A value owned by an `Rc` pointer is immutable. Suppose you try to add some text to the end of the string:

In [50]:
s.push_str(" noodles");

Error: cannot borrow data in an `Rc` as mutable

Rust’s memory and thread-safety guarantees depend on ensuring that no value is ever simultaneously shared and mutable. Rust assumes the referent of an `Rc` pointer might in general be shared, so it must not be mutable.

One well-known problem with using reference counts to manage memory is that, if there are ever two reference-counted values that point to each other, each will hold the other’s reference count above zero, so the values will never be freed

![image.png](asset/ch4/8.png)

It is possible to leak values in Rust this way, but such situations are rare. You cannot create a cycle without, at some point, making an older value point to a newer value. This obviously requires the older value to be mutable. Since `Rc` pointers hold their referents immutable, it’s not normally possible to create a cycle.

这段话如何理解：Rust不允许未初始化的值，也就是说，old的对象指向的一定是一个已存在的值（更old的对象），也就是上图中从右向左的指针是可以存在的（认为左边的对象是更old的）。但是想要构成循环引用，就需要让old的对象的指针指向新的对象，但是此时old的指针已经指向了更old的对象了；已知Rc指针是immutable的，所以原则来说，在Rust的规则下，循环引用不可能存在。

However, Rust does provide ways to create mutable portions of otherwise immutable values; this is called _interior mutability_, and we cover it in “Interior Mutability” on page 225. If you com‐ bine those techniques with Rc pointers, you can create a cycle and leak memory.

You can sometimes avoid creating cycles of Rc pointers by using weak pointers, `std::rc::Weak`, for some of the links instead. However, we won’t cover those in this book; see the standard library’s documentation for details.